In [1]:
# Required imports
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Load the dataset
train_data = pd.read_excel("/content/drive/MyDrive/DSW/train_data.xlsx")
test_data = pd.read_excel("/content/drive/MyDrive/DSW/test_data.xlsx")

# Inspect the dataset
print("Train Data Info:")
print(train_data.info())

print("\nTest Data Info:")
print(test_data.info())


Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113705 entries, 0 to 113704
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   customer_id          113705 non-null  int64         
 1   transaction_date     113705 non-null  datetime64[ns]
 2   sub_grade            113705 non-null  object        
 3   term                 113705 non-null  object        
 4   home_ownership       113705 non-null  object        
 5   cibil_score          113705 non-null  int64         
 6   total_no_of_acc      113705 non-null  int64         
 7   annual_inc           113705 non-null  float64       
 8   int_rate             113705 non-null  float64       
 9   purpose              113705 non-null  object        
 10  loan_amnt            113705 non-null  int64         
 11  application_type     113705 non-null  object        
 12  installment          113705 non-null  float64       
 1

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

class XGBoostModel:
    def __init__(self):
        self.model = xgb.XGBClassifier(random_state=42)
    
    def load(self, train_df, test_df):
        # Dropping unnecessary columns as it makes the overfitting problem
        unnecessary_columns = ['customer_id', 'transaction_date']
        train_df = train_df.drop(columns=unnecessary_columns, errors='ignore')
        test_df = test_df.drop(columns=unnecessary_columns, errors='ignore')

        # Separate features and target
        self.X_train = train_df.drop(columns=['loan_status'])
        self.y_train = train_df['loan_status']
        self.X_test = test_df.drop(columns=['loan_status'])
        self.y_test = test_df['loan_status']
        print("Data loaded successfully.")

    def preprocess(self):
        # Handle outliers by capping values
        numeric_cols = self.X_train.select_dtypes(include=np.number).columns
        for col in numeric_cols:
            upper_limit = self.X_train[col].quantile(0.99)
            lower_limit = self.X_train[col].quantile(0.01)
            self.X_train[col] = np.clip(self.X_train[col], lower_limit, upper_limit)
            self.X_test[col] = np.clip(self.X_test[col], lower_limit, upper_limit)

        # One-hot encode for encoding the categorical columns
        categorical_cols = self.X_train.select_dtypes(include='object').columns
        self.X_train = pd.get_dummies(self.X_train, columns=categorical_cols, drop_first=True)
        self.X_test = pd.get_dummies(self.X_test, columns=categorical_cols, drop_first=True)

        # Align test set columns with train set
        self.X_test = self.X_test.reindex(columns=self.X_train.columns, fill_value=0)
        print("Data preprocessed successfully.")


    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)
        print("XGBoost Model training complete.")

    def test(self, X_test, y_test):
        # Predictions and evaluation
        predictions = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print("Accuracy on test set:", accuracy)
        print("\nClassification Report:\n", classification_report(y_test, predictions))
        print("\nConfusion Matrix:\n", confusion_matrix(y_test, predictions))

    def predict(self, new_data, X_train_columns):
        # Align new data with the train set columns
        new_data = pd.get_dummies(new_data, columns=new_data.select_dtypes(include='object').columns, drop_first=True)
        new_data = new_data.reindex(columns=X_train_columns, fill_value=0)
        return self.model.predict(new_data)


In [19]:
# Instantiate the XGBoost Model
xgb_model = XGBoostModel()

# Train the model
xgb_model.train(rf_model.X_train, rf_model.y_train)  # Use preprocessed data from earlier

# Test the model
xgb_model.test(rf_model.X_test, rf_model.y_test)


XGBoost Model training complete.
Accuracy on test set: 0.6807806031933767

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.27      0.38      3055
           1       0.69      0.91      0.79      5400

    accuracy                           0.68      8455
   macro avg       0.66      0.59      0.58      8455
weighted avg       0.67      0.68      0.64      8455


Confusion Matrix:
 [[ 818 2237]
 [ 462 4938]]


In [20]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

class LightGBMModel:
    def __init__(self):
        self.model = lgb.LGBMClassifier(random_state=42)

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)
        print("LightGBM Model training complete.")

    def test(self, X_test, y_test):
        # Predictions and evaluation
        predictions = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print("Accuracy on test set:", accuracy)
        print("\nClassification Report:\n", classification_report(y_test, predictions))
        print("\nConfusion Matrix:\n", confusion_matrix(y_test, predictions))

    def predict(self, new_data, X_train_columns):
        # Align new data with the train set columns
        new_data = pd.get_dummies(new_data, columns=new_data.select_dtypes(include='object').columns, drop_first=True)
        new_data = new_data.reindex(columns=X_train_columns, fill_value=0)
        return self.model.predict(new_data)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [21]:
# Instantiate the LightGBM Model
lgbm_model = LightGBMModel()

# Train the model
lgbm_model.train(rf_model.X_train, rf_model.y_train)  # Use preprocessed data from earlier

# Test the model
lgbm_model.test(rf_model.X_test, rf_model.y_test)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 84016, number of negative: 29689
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1233
[LightGBM] [Info] Number of data points in the train set: 113705, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.738895 -> initscore=1.040231
[LightGBM] [Info] Start training from score 1.040231
LightGBM Model training complete.
Accuracy on test set: 0.6777054997043169

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.24      0.35      3055
           1       0.68      0.92      0.79      5400

    accuracy                           0.68      8455
   macro avg       0.66      0.58      0.57  